In [9]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from io import StringIO
import random 
import pickle
import os
from dotenv import load_dotenv

load_dotenv()


True

In [3]:
# Loading graph after summary generation
with open('graph.pkl', 'rb') as f:
    G = pickle.load(f)

In [16]:
import requests
import json
import datetime
from groq import Groq

client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# Function to get relevant events before a specific date
def get_events_before_date(actor, recipient, date):
    date = datetime.datetime.strptime(date, "%Y-%m-%d")
    relevant_events = []

    # Iterate through nodes in the graph
    for node, attributes in G.nodes(data=True):
        if "event_date" in attributes:
            event_date = datetime.datetime.strptime(attributes["event_date"], "%Y-%m-%d")
            if event_date < date:
                # Check if actor or recipient is involved
                for neighbor in G.neighbors(node):
                    if neighbor == actor or neighbor == recipient:
                        relevant_events.append({
                            "event_node": node,
                            "event_date": attributes.get("event_date"),
                            "event_type": attributes.get("event_type"),
                            "quad_code": attributes.get("quad_code"),
                            "contexts": attributes.get("contexts"),
                            "summary": attributes.get("summary"),
                            "relation_with": neighbor,
                        })

    return relevant_events

# Function to interact with the Groq API
def query_groq(prompt):

    groq_input = (prompt)
    try:
        # Call the Groq API to generate a summary
        chat_completion = client.chat.completions.create(
            messages=[{"role": "user", "content": groq_input}],
            model="llama3-8b-8192",
        )

        return chat_completion.choices[0].message.content
    except Exception as e:
        print(f"Error querying Groq API: {e}")
        return None

# Main function to predict relations
def predict_relations_with_llm(actor, recipient, date):
    # Step 1: Gather relevant events
    events = get_events_before_date(actor, recipient, date)

    if not events:
        print(f"No relevant events found for {actor} and {recipient} before {date}.")
        return

    # Step 2: Prepare a structured prompt for the LLM
    event_details = "\n".join(
        [
            f"Event Node: {event['event_node']}, Date: {event['event_date']}, Event Type: {event['event_type']}, "
            f"Quad Code: {event['quad_code']}, Contexts: {event['contexts']}, Summary: {event['summary']}, "
            f"Relation with: {event['relation_with']}"
            for event in events
        ]
    )

    prompt = f"""
You are a knowledge graph analysis expert. Based on the following event data, predict the top 3 possible relations between the actor '{actor}' and the recipient '{recipient}' before the date '{date}'.

Event Details:
{event_details}

Consider the summaries, event type, quad code, and context. Provide your top 3 ranked predictions for the potential relation between the entities ONLY FROM: ASSAULT, AID, ACCUSE, REQUEST, PROTEST, COERCE, RETREAT, MOBILIZE, SANCTION, THREATEN AND GIVE REASONING
    """

    # Step 3: Query the Groq API
    print("\nQuerying Groq API for predictions...\n")
    prediction = query_groq(prompt)

    # Step 4: Output the results
    if prediction:
        print(f"Predicted Relations:\n{prediction}")
    else:
        print("Failed to retrieve predictions from the Groq API.")



In [17]:
predict_relations_with_llm("Narendra Modi", "Congee", "2024-12-01")



Querying Groq API for predictions...

Predicted Relations:
Based on the event data and summaries, I predict the top 3 possible relations between the actor 'Narendra Modi' and the recipient 'Congee' before the date '2024-12-01' as follows:

1. **AID**: There is no specific event mentioning Congee in the provided data. However, considering Narendra Modi's involvement in a verbal conflict with different political entities on May 26, 2024 (E9, E52, E88), it's possible that he might have used his influence to provide aid to a faction or group that shares similar political views, including those who might be involved in protests or demonstrations. However, this prediction is speculative, and I base it on the assumption that Narendra Modi's involvement in verbal conflicts could potentially lead to his support for a particular group or faction.

2. **PROTEST**: Although there is no direct event mentioning Congee in the provided data, Narendra Modi's involvement in a protest-turned-violent-con